In [2]:
#Example 1

#Data import from online storage and copy saved to local machine

import pandas as pd

url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'

data = pd.read_csv(url)

data.to_csv('test_Melbourne')

data.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [7]:
#Example 2

#Read data from BigQuery

#Process the data in local Jupyter notebook

#Save the results to BigQuery

import pandas as pd #Also: python -m pip install pandas_gbq

#from google.oauth2 import service_account #Uncomment if not using token

import pandas_gbq

#credentials = service_account.Credentials.from_service_account_file('path_to_json') #Uncomment and add corerct path if not using token

projectid = "decisive-circle-378616" #Use your own id!

#Download the data

sql = """
SELECT * FROM `bigquery-public-data.world_bank_global_population.population_by_country` 
"""

#df_raw = pd.read_gbq(sql,projectid,credentials=credentials) #Export to local machine, service account key used

df_raw = pd.read_gbq(sql,projectid) #Export to local machine, token used

##### Data processing
df_tr = df_raw.T #Transpose

#1 Column headers are not ok

df_tr.columns = df_tr.iloc[0] #Rename column headers

#2 Useless rows, (1,2)

df_tr = df_tr.drop(df_tr.index[[0,1]])

#Time inofrmation not in a good format, fix.

df_tr.reset_index(inplace=True)

df_tr.columns.name = None

df_tr = df_tr.rename(columns = {'index':'year'})

df_tr['year'] = pd.to_numeric(df_tr['year'].str[5:9])

df_tr.drop(df_tr.tail(1).index,inplace=True) 

#Select columns and data

df_tr = df_tr[['year','Finland','Sweden','Norway','Denmark','Iceland']]

df_tr = df_tr.astype(int)

#df_tr.head()

#df_tr.info()

df_tr.to_gbq('test_data_2023.population_2023', project_id='decisive-circle-378616', if_exists = 'replace')

In [34]:
#Example 3

#Query and process the data in cloud and then save it there.

from google.cloud import bigquery
from google.oauth2 import service_account

# Construct a BigQuery client object.

credentials = service_account.Credentials.from_service_account_file('C:/Users/eelke/OAMK/Data analysis/decisive-circle-378616-2972ceec92b5.json')

client = bigquery.Client(credentials=credentials) #Service account key

#client = bigquery.Client(location="US", project="decisive-circle-378616") #Token

#Set table_id to the ID of the destination table.

table_id = "decisive-circle-378616.test_data_2023.shakespeare_2023"

job_config = bigquery.QueryJobConfig(destination=table_id)

sql = """
SELECT corpus
FROM `bigquery-public-data.samples.shakespeare`
GROUP BY corpus;"""

# Start the query, passing in the extra configuration.

query_job = client.query(sql, job_config=job_config)  # Make an API request.

query_job.result()  # Wait for the job to complete.


print("Query results loaded to the table {}".format(table_id))

Query results loaded to the table decisive-circle-378616.test_data_2023.shakespeare_2023
